## Prueba de concepto sobre segmentacion de abstracts con NLP
- 

#### Mapear los archivos

In [12]:
import os
# import sys
import re
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET

from sklearn.feature_extraction.text import CountVectorizer

path = 'C:/Users/user/Desktop/Globant/T1/2020/'
# sys.path.append(path)

ModuleNotFoundError: No module named 'sklearn'

In [2]:
#TODO: Dar un comentario a cada funcion
def abstract_names_function(path):
    """
    Lista todos los nombres de los archivos en el directorio especificado.

    Args:
    path (str): La ruta al directorio del que se quieren listar los archivos.

    Returns:
    list: Una lista con los nombres de todos los archivos en el directorio.
    """
    # Verificar si el directorio existe
    if not os.path.exists(path):
        return "El directorio no existe."
    
    # Obtener todos los archivos y subdirectorios en el directorio especificado
    nombres_de_archivos = [archivo for archivo in os.listdir(path) if os.path.isfile(os.path.join(path, archivo))]

    return nombres_de_archivos

def abstract_sampler(full_list, n):
    """
    Selecciona una muestra aleatoria de n elementos de una lista de strings.

    Args:
    full_list (list): La lista original de strings de la cual seleccionar la muestra.
    n (int): El número de elementos a seleccionar.

    Returns:
    list: Una lista con n elementos seleccionados aleatoriamente de la lista original.
    """
    # Verificar si n es mayor que la longitud de la lista
    if n > len(full_list):
        return "Error: n es mayor que el número de elementos en la lista."
    
    # Seleccionar n elementos aleatorios de la lista
    muestra_seleccionada = random.sample(full_list, n)
    
    return muestra_seleccionada

def extract_data(file_path):
    """Función para extraer los datos relevantes de un archivo XML

    Args:
        file_path (arr): array con los nombres de los archivos en string

    Returns:
        _dict_: devuelve un diccionario con el titulo y el resumen del archivo
    """
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Extraer información básica
    title = root.find('.//AwardTitle').text if root.find('.//AwardTitle') is not None else "No Title"
    abstract = root.find('.//AbstractNarration').text if root.find('.//AbstractNarration') is not None else "No Abstract"
    
    return {
        "title": title,
        "abstract": abstract
    }

def clean_text(text): #TODO: Terminar documentacion
    """Función para limpiar el texto de los resúmenes

    Args:
        text (_type_): _description_

    Returns:
        _type_: _description_
    """
    if pd.isna(text):
        return ""  # Retorna un string vacío si el texto es NaN

    # Eliminar etiquetas HTML
    text = re.sub(r'<[^>]+>', '', text)
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar espacios adicionales
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [3]:
# Ejemplo de uso:
archivos_full = abstract_names_function(path)
print(len(archivos_full))

13300


In [4]:
# Muestreo de la data
n = 1000 ## Cerca del 7% de los datos

muestra = abstract_sampler(archivos_full, n)

print("archivos: ", len(muestra))
print("array_view: ",muestra[:10])

archivos:  1000
array_view:  ['2010237.xml', '2024238.xml', '2027532.xml', '2047665.xml', '2019074.xml', '2028920.xml', '2004110.xml', '2040575.xml', '2048221.xml', '2018626.xml']


In [5]:
path + muestra[0]

'C:/Users/user/Desktop/Globant/T1/2020/2010237.xml'

#### Ingestar la muestra

In [8]:
# Procesar cada archivo y almacenar los resultados en un DataFrame
data_list = [extract_data(path+file) for file in muestra]
df_abstracts = pd.DataFrame(data_list)

display(df_abstracts.head())

,title,abstract
0,Collaborative Research: Design Talks: Building...,Inclusion of engineering design activities in ...
1,Institutional Transformation: VERITIES - Virtu...,This award supports a research project to tran...
2,Collaborative Research: Flying snakes: fluid m...,There are numerous forms of flight ranging fro...
3,CAREER: A Keystone for Precision Neutrino Phys...,One of the major intellectual achievements of ...
4,MRI: Acquisition of a 400 MHz Nuclear Magnetic...,This award is supported by the Major Research ...


In [9]:
# Aplicar la limpieza a los resúmenes
df_abstracts['clean_abstract'] = df_abstracts['abstract'].apply(clean_text)

display(df_abstracts.head())

,title,abstract,clean_abstract
0,Collaborative Research: Design Talks: Building...,Inclusion of engineering design activities in ...,inclusion of engineering design activities in ...
1,Institutional Transformation: VERITIES - Virtu...,This award supports a research project to tran...,this award supports a research project to tran...
2,Collaborative Research: Flying snakes: fluid m...,There are numerous forms of flight ranging fro...,there are numerous forms of flight ranging fro...
3,CAREER: A Keystone for Precision Neutrino Phys...,One of the major intellectual achievements of ...,one of the major intellectual achievements of ...
4,MRI: Acquisition of a 400 MHz Nuclear Magnetic...,This award is supported by the Major Research ...,this award is supported by the major research ...


#### EDA Simple

In [ ]:


# Configurando CountVectorizer para extraer las palabras más frecuentes
vectorizer = CountVectorizer(stop_words='english', max_features=20)
data_vectorized = vectorizer.fit_transform(df_abstracts['clean_abstract'])

# Sumar las frecuencias de cada palabra
sum_words = data_vectorized.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

# Crear un DataFrame con las palabras más comunes
df_words = pd.DataFrame(words_freq, columns=['word', 'freq'])

# Visualización de las palabras más comunes
plt.figure(figsize=(10, 6))
sns.barplot(x='freq', y='word', data=df_words)
plt.title('Top 20 palabras más frecuentes en los resúmenes')
plt.xlabel('Frecuencia')
plt.ylabel('Palabra')
plt.show()
